In [ ]:
import duckdb
import psycopg2
import pandas as pd

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname='postgres',  # Your dbname is 'test'
    user='postgres',
    password='123456',
    host='localhost',
    port='5432'  # Default PostgreSQL port
)

# Connect to DuckDB
con = duckdb.connect()

# List of table names
table_names = ['restaurant', 'brand', 'menu', 'consumption_record','coupon','points','coupon_record','member','reservation_record','feedback_record']#,'compose']

# Load data from PostgreSQL into DuckDB
for table_name in table_names:
    query_str = f"SELECT * FROM {table_name}"
    df = pd.read_sql_query(query_str, conn)
    # Create a real table in DuckDB
    con.execute(f"CREATE TABLE IF NOT EXISTS {table_name} AS SELECT * FROM df")  # Create table and insert data

# Close the PostgreSQL connection
conn.close()

# Function to execute a query in DuckDB
def query(query_str):
    result = con.execute(query_str).fetchall()
    column_names = [desc[0] for desc in con.description]
    return column_names, result




import socket
import threading
import signal
import sys

# 設定伺服器的IP和端口
host = '127.0.0.1'  # 本地端IP
port = int(input('port:'))  # 端口

# 創建 TCP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# 綁定IP和端口
server_socket.bind((host, port))

# 開始監聽連接
server_socket.listen(5)
print(f"Server is listening on {host}:{port}...")

# 儲存所有 client 的連接
clients = []

# 儲存每個 client 的角色
client_roles = {}

# 是已登入member的話存member id
member = {}

# 控制伺服器是否繼續運行
running = True

# 儲存伺服器的關閉信號處理
def stop_server():
    global running
    running = False
    print("\nShutting down the server...")

    # 關閉所有 client 連接
    for client_socket in clients:
        try:
            client_socket.close()
        except Exception as e:
            print(f"Error while closing client socket: {e}")

    # 關閉伺服器的 socket
    server_socket.close()
    print("Server socket closed.")

# 顯示當前在線人數
def display_online_count():
    print(f"Current online users: {len(clients)}")

# 選擇角色的函數
def choose_role(client_socket):
    role = client_socket.recv(1024).decode()

    # 確保客戶端選擇了有效的角色
    while role not in ['1', '2', '3', '4']:
        client_socket.send("Invalid choice. Please select a role from 1 to 4: \n1. Member\n2. Sub-brand Store Staff\n3. Sub-brand Store Manager\n4. Marketing and Operations Manager\nType 'exit' to leave.".encode())
        role = client_socket.recv(1024).decode()

    # 根據選擇返回角色名稱
    if role == '1':
        return "Member"
    elif role == '2':
        return "Sub-brand Store Staff"
    elif role == '3':
        return "Sub-brand Store Manager"
    elif role == '4':
        return "Marketing and Operations Manager"





def member_move(client_socket,member_id):
    message = client_socket.recv(1024).decode()

    # 確保客戶端選擇了有效的角色
    while message not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
        client_socket.send("Invalid choice.".encode())
        client_socket.send("請輸入1到6之間的數字".encode())
        message = client_socket.recv(1024).decode()

    # 根據選擇返回角色名稱
    if message == '1':
        # 會員可以查詢自己擁有的優惠券清單
        return f'''
                SELECT
                b. brand_name AS 品牌名稱 ,
                c. coupon_name AS 優惠券名稱 ,
                c. description AS 優惠券描述 ,
                c. due_date AS 到期日期,
                CASE
                    WHEN cr.consumption_record_id IS NULL THEN '尚未使用'
                    ELSE '已使用' END AS 使用狀況
                FROM
                coupon c
                JOIN
                coupon_record cr ON c. coupon_id = cr. coupon_id
                join
                brand b on b.brand_id=c.brand_id
                WHERE
                cr.Member_id = '{member_id}'
                AND c. due_date >= CURRENT_DATE
                ORDER BY
                c. due_date ASC;
                '''
    elif message == '2':
        # 查詢Points可兌換商品列表
        return f'''
                SELECT
                Item_name AS 商品名稱 ,
                Price AS 所需點數 ,
                Availability AS 商品供應狀態,
                description AS 商品描述
                FROM
                MENU
                WHERE
                Menu_Category = '兌換商品';
                '''
    elif message == '3':
        # 查詢消費紀錄
        return f'''
                SELECT
                b.brand_name AS 品牌,
                r.restaurant_name AS 分店,
                cr.datetime AS 消費日期 ,
                cr.Amount AS 消費金額
                FROM
                CONSUMPTION_RECORD cr
                join restaurant r on r.restaurant_id=cr.restaurant_id
                join brand b on r.brand_id=b.brand_id
                WHERE
                Member_id = '{member_id}'
                ORDER BY
                datetime DESC ;
                '''
    elif message == '4':
        #查詢點數紀錄
        return f'''
                SELECT
                p.Created_At AS 日期 ,
                b.brand_name AS 品牌,
                r.restaurant_name AS 分店,
                p.points_change AS 點數變動 ,
                p.points_remark AS 備註 ,
                cr.Amount AS 當日消費金額
                FROM points p
                left join consumption_record cr on cr.consumption_record_id=p.consumption_record_id
                left join restaurant r on r.restaurant_id=cr.restaurant_id
                left join brand b on r.brand_id=b.brand_id
                WHERE
                p.Member_id = '{member_id}'
                ORDER BY
                datetime DESC ;
                '''
    elif message == '5':
        # 查詢所有有營業的分店位置
        # 阪前和牛鐵板燒
        client_socket.send("ok".encode())
        client_socket.send("brand name:".encode())
        restaurant=client_socket.recv(1024).decode()
        return f'''
                SELECT
                b.brand_name AS 品牌,
                r.Address AS 分店地址
                FROM
                RESTAURANT r
                join brand b on r.brand_id=b.brand_id
                WHERE
                r.Status = 'open' and
                b.brand_name='{restaurant}'
                ORDER BY
                r.area ASC;
                '''
    elif message == '6':
        return f'''
                select
                name,  birthdate,points, email
                from member where member_id='{member_id}'


                '''
    elif message == '7':
        return f'''


                '''
    elif message == '8':
        return f'''


                '''
    elif message == '9':
        return f'''



                '''
restaurant_id=0
def staff_move(client_socket,restaurant_id):
    message = client_socket.recv(1024).decode()

    # 確保客戶端選擇了有效的角色
    while message not in ['1', '2', '3']:
        client_socket.send("Invalid choice.".encode())
        client_socket.send("請輸入1 or 2 or 3".encode())
        message = client_socket.recv(1024).decode()
# 101
    # 根據選擇返回角色名稱
    if message == '1':
        # 新增消費紀錄
        client_socket.send("\n正在增添一筆消費紀錄".encode())
        client_socket.send("請輸入用餐會員ID:".encode())
        member_id=client_socket.recv(1024).decode()
        client_socket.send(f"\n謝謝{member_id}會員來訪".encode())
        client_socket.send("本次消費金額是:".encode())
        amount=client_socket.recv(1024).decode()

        client_socket.send(f"\n{member_id}的消費金額是{amount}元".encode())
        client_socket.send("請問有使用優惠券嗎?有則輸入優惠券ID，無則輸入N即可".encode())
        couponID=client_socket.recv(1024).decode()


        max_id_query = """
            SELECT MAX(Consumption_Record_ID) FROM Consumption_Record
        """
        max_id_result = con.execute(max_id_query).fetchone()

        next_id = (max_id_result[0] or 0) + 1  # If no records exist, start from 1
        print(next_id)
        print(couponID,next_id,restaurant_id,member_id)
        if couponID!='N': # coupon 1
#            con.execute(command).fetchall()
            return f'''

                    -- 更新優惠券紀錄
                    UPDATE coupon_record
                    SET Consumption_Record_ID = {next_id}
                    WHERE coupon_record_id = (
                        SELECT coupon_record_id
                        FROM coupon_record
                        WHERE member_id = '{member_id}'
                          AND coupon_id = {couponID}
                          AND Consumption_Record_ID IS NULL
                        LIMIT 1
                    );
                    -- 插入新的消費紀錄，並且取得插入的 ID
                    INSERT INTO Consumption_Record (Consumption_Record_ID, Amount, Member_ID, Restaurant_ID, DateTime)
                    VALUES ({next_id}, {amount}, '{member_id}', {restaurant_id}, CAST(LOCALTIMESTAMP AS TIMESTAMP(0)))
                    returning *;
                    '''
        else: #'1980-12-08 22:50:00'
            return f'''
                    INSERT INTO CONSUMPTION_RECORD (Consumption_Record_ID,Restaurant_id, Member_id   , datetime, Amount )
                    VALUES ({next_id}, {restaurant_id}, '{member_id}'  ,CAST(LOCALTIMESTAMP AS TIMESTAMP(0)),{amount})
                    RETURNING *;
                    '''
    elif message == '2':
        # 查詢消費紀錄
        return f'''
                -- 查詢消費紀錄
                select
                cs.datetime AS 時間,
                cs.amount AS 金額,
                cr.coupon_id AS 使用優惠券
                from Consumption_Record cs
                left join coupon_record cr on cs.Consumption_Record_ID=cr.Consumption_Record_ID
                where cs.restaurant_id={restaurant_id}


                '''
    elif message == '3':
        #看訂位
        client_socket.send("正在查詢訂位紀錄".encode())
        client_socket.send("請輸入會員ID:".encode())
        member_id=client_socket.recv(1024).decode()
        return f'''


                SELECT
                Guest_cnt AS 人數 ,
                Datetime AS 訂位時間
                FROM
                RESERVATION_RECORD
                WHERE
                Member_id = '{member_id}'
                and Restaurant_id={restaurant_id}
                ORDER BY
                Datetime DESC ;

                '''


def store_manager_move(client_socket):
    message = client_socket.recv(1024).decode()

    # 確保客戶端選擇了有效的角色
    while message not in ['1', '2']:
        client_socket.send("Invalid choice.".encode())
        client_socket.send("請輸入1 or 2".encode())
        message = client_socket.recv(1024).decode()

    # 根據選擇返回角色名稱
    if message == '1':
        #修改會員點數
        client_socket.send("\n".encode())
        client_socket.send("輸入會員ID:".encode())
        member_id=client_socket.recv(1024).decode()


        query = f"""
            SELECT points FROM member WHERE Member_id = '{member_id}';
        """
        points = con.execute(query).fetchone()

        client_socket.send(f"會員現有點數: {points[0]}".encode())
        client_socket.send("您要加/扣多少點數?".encode())
        adjust_points=client_socket.recv(1024).decode()



        max_id_query = """
            SELECT MAX(Points_ID) FROM points
        """
        max_id_result = con.execute(max_id_query).fetchone()

        next_id = (max_id_result[0] or 0) + 1  # If no records exist, start from 1
        #回傳該會員的點數變動紀錄
        return f'''
                UPDATE MEMBER
                SET
                Points = Points+{adjust_points}
                WHERE
                Member_id = '{member_id}';

                -- 記 錄 點 數 變 動 明 細
                INSERT INTO POINTS ( points_change , points_Remark ,
                consumption_record_id , Member_id ,Created_At,Points_ID)
                VALUES ({adjust_points} , '經理直接操作' , NULL ,'{member_id}',CAST(LOCALTIMESTAMP AS TIMESTAMP(0)),{next_id})
                returning *;

                '''
    elif message == '2':
        return f'''

        -- 查 詢 指 定 期 間 的 營 業 額
        SELECT
        COUNT ( consumption_record_id ) AS 次數,
        restaurant_id AS 餐廳 ,
        SUM( amount ) AS 營業額
        FROM
        consumption_record
        WHERE
        datetime >= '2024-01-01' AND datetime <= '2024-12-31'
        GROUP BY
        restaurant_id
        ORDER BY
        營業額 DESC ;
                '''

def market_manager_move(client_socket):
    message = client_socket.recv(1024).decode()

    # 確保客戶端選擇了有效的角色
    while message not in ['1', '2', '3', '4']:
        client_socket.send("Invalid choice.".encode())
        client_socket.send("請輸入1到4之間的數字".encode())
        message = client_socket.recv(1024).decode()

    # 根據選擇返回角色名稱
    if message == '1':
        client_socket.send("你想查詢哪位顧客的回饋呢?".encode())
        client_socket.send("請輸入會員ID:".encode())
        member_id=client_socket.recv(1024).decode()
        return f'''

                select
                c.datetime AS 日期 ,
                b.brand_name AS 品牌,
                r.restaurant_name AS 分店,
                f.content AS 回饋內容,
                c.amount AS 消費金額
                from feedback_record f
                join consumption_record c on c.consumption_record_id=f.consumption_record_id
                join restaurant r on r.restaurant_id=c.restaurant_id
                join brand b on r.brand_id=b.brand_id
                where c.member_id='{member_id}'
                order by
                c.datetime desc
                '''
    elif message == '2':
        #新增品牌

        max_id_query = """
            SELECT MAX(Brand_ID) FROM brand
        """
        max_id_result = con.execute(max_id_query).fetchone()

        next_id = (max_id_result[0] or 0) + 1  # If no records exist, start from 1

        client_socket.send("你想新增甚麼品牌呢?".encode())
        client_socket.send("請輸入品牌名稱:".encode())
        brand_name=client_socket.recv(1024).decode() #汁料庫
        client_socket.send("此品牌是哪一年創辦的呢?".encode())
        client_socket.send("請輸入年份:".encode())
        year=client_socket.recv(1024).decode() #汁料庫
        return f'''
                -- 新 增 品 牌 ：
                INSERT INTO BRAND ( Brand_name , founded_year , Status ,brand_id)
                VALUES ('{brand_name}', {year} , 'active',{next_id})
                returning *;
                '''
    elif message == '3':
        # 使品牌歇業
        client_socket.send("哪個品牌歇業了呢?".encode())
        client_socket.send("請輸入品牌名稱:".encode())
        brand_name=client_socket.recv(1024).decode() #汁料庫
        return f'''
                UPDATE BRAND
                SET
                Status = 'Inactive'
                WHERE
                Brand_name = '{brand_name}'
                RETURNING Brand_name, Status;
                '''
    elif message == '4':
        client_socket.send("你想查詢active還是inactive的品牌?".encode())
        client_socket.send("1. active \n2. inactive".encode())
        active=client_socket.recv(1024).decode() #汁料庫
        if active==1:
            active='Active'
        else:
            active='Inactive'
        return f'''
                select brand_name AS 品牌,
                founded_year AS 創辦年,
                status AS 營業狀況
                from brand
                where status='{active}'
                '''

def display(query_str):
    print("displaysuccess\n")
    # Execute the query and load the results into a DataFrame
    columns, data = query(query_str)
    df = pd.DataFrame(data, columns=columns)
    return df.to_string(index=False)  # Or use df.to_json() if you prefer JSON format








system_login=[False,False,False]
def system_pass(system_password):
    if system_password=='starwars':
        return True
    else:
        return False


def exit(message):
    # 若 client 傳送 "exit"，則結束連接
    if message.lower() == "exit":
        print(f"Client {client_address} has exited.")

#    print(f"Received from client {client_address}: {message}")


def logged_in(member_id,password):
    query_str=f'''
    select member_id
    from member
    where member_id='{member_id}' and password='{password}'
    '''
    columns, data = query(query_str)
    if len(data)>0:
        return True
    else:
        return False

# 處理每個 client 連接的函數
def handle_client(client_socket, client_address):
    print(f"Connection from {client_address} has been established.")

    try:
        while running:


            # 如果該 client 還沒有選擇過角色，提示其選擇角色
            if client_address not in client_roles:
                client_socket.send("請選擇您的角色: \n1. 會員\n2. 員工\n3. 品牌經理\n4. 營運經理\nReminder: Type 'exit' to leave.\n\nYour choice:".encode())
                role_name = choose_role(client_socket)  # 如果還沒有選擇角色，提示選擇
                # 存儲該客戶端的角色
                client_roles[client_address] = role_name
                print(f"Client {client_address} has selected the role: {role_name}")
                client_socket.send(f"\n歡迎進入系統\n".encode())
            else: # 如果已經選擇過角色，則直接取得角色

                # 根據角色處理不同的邏輯
                if client_roles[client_address] == "Member":
                    if client_address not in member:
                        # 處理會員角色的邏輯 (可以根據需要填充)
                        client_socket.send(f"請輸入您的會員ID:".encode())
                        member_id = client_socket.recv(1024).decode()
                        client_socket.send(f"\n{member_id}您好!".encode())
                        client_socket.send(f"請輸入您的密碼:".encode())
                        password = client_socket.recv(1024).decode()
                        if logged_in(member_id,password):
                            member[client_address]=member_id
                            client_socket.send("\n您已成功登入\n".encode())
                        else:
                            client_socket.send("login fail, plz try again".encode())


                    else:
                        client_socket.send(f"{member[client_address]}您好! 請問您想:\n1. 查詢擁有的優惠券\n2. 查詢點數可兌換之商品列表\n3. 查詢消費紀錄\n4. 查詢點數紀錄\n5. 查詢所有分店及位置\n6. 個人資訊\n\nYour choice:".encode())
                        query_str=member_move(client_socket,member[client_address]) # 9個選項
                        client_socket.send(f'\n\n{display(query_str)}\n\n'.encode())

                elif client_roles[client_address] == "Sub-brand Store Staff":







                    if system_login[0]:
                        # 處理子品牌店員角色的邏輯 (可以根據需要填充)
                        client_socket.send(f"{restaurant_id}餐廳員工您好! 請問您想: \n1. 增添一筆消費紀錄\n2. 查詢{restaurant_id}餐廳消費紀錄\n3. 查詢某會員訂位紀錄\n\nYour choice:".encode())
                        query_str =staff_move(client_socket,restaurant_id) # 3個選項
                        client_socket.send(f'\n\n{display(query_str)}\n\n'.encode())
                    else:
                        client_socket.send(f"請輸入您所在餐廳的ID:".encode())
                        restaurant_id = client_socket.recv(1024).decode()
                        client_socket.send(f"\n{restaurant_id}餐廳員工您好".encode())
                        client_socket.send("請輸入系統密碼:".encode())
                        system_password = client_socket.recv(1024).decode()
                        if system_pass(system_password):
                            client_socket.send("\n密碼正確\n".encode())
                            system_login[0]=True
                        else:
                            client_socket.send("incorrect, please try again\n".encode())

                elif client_roles[client_address] == "Sub-brand Store Manager":

                    if system_login[1]:
                        # 處理子品牌店經理角色的邏輯 (可以根據需要填充)
                        client_socket.send(f"品牌經理您好! 請問您想: \n1. 修改某會員點數\n2. 查詢各餐廳的營業額\n\nYour choice:".encode())
                        query_str =store_manager_move(client_socket) # 2個選項
                        client_socket.send(f'\n\n{display(query_str)}\n\n'.encode())
                    else:
                        client_socket.send("請輸入系統密碼:".encode())
                        system_password = client_socket.recv(1024).decode()
                        if system_pass(system_password):
                            client_socket.send("\n密碼正確\n".encode())
                            system_login[1]=True
                        else:
                            client_socket.send("incorrect, please try again\n".encode())


                elif client_roles[client_address] == "Marketing and Operations Manager":

                    if system_login[2]:
                        # 處理行銷及營運管理人員角色的邏輯 (可以根據需要填充)
                        client_socket.send(f"營運經理您好! 請問您想: \n1. 查詢顧客回饋\n2. 新增品牌資訊\n3. 將歇業品牌資訊更新\n4. 查看所有品牌\n\nYour choice:".encode())
                        query_str =market_manager_move(client_socket) # 4個選項
                        client_socket.send(f'\n\n{display(query_str)}\n\n'.encode())
                    else:
                        client_socket.send("請輸入系統密碼:".encode())
                        system_password = client_socket.recv(1024).decode()
                        if system_pass(system_password):
                            client_socket.send("\n密碼正確\n".encode())
                            system_login[2]=True
                        else:
                            client_socket.send("incorrect, please try again\n".encode())

    except Exception as e:
        print(e)
#        print(f"Error while handling client {client_address}: {e}")

    finally:
        # 關閉與 client 的連接
        client_socket.close()
        print(f"Connection with {client_address} closed.")
        clients.remove(client_socket)
        display_online_count()  # 顯示當前在線人數

# 接受並處理客戶端連接
def accept_connections():
    while running:
        try:
            # 接受客戶端的連接
            client_socket, client_address = server_socket.accept()

            # 將 client socket 加入到 clients 列表
            clients.append(client_socket)

            # 顯示當前在線人數
            display_online_count()

            # 為每個 client 創建一個新的 thread
            client_thread = threading.Thread(target=handle_client, args=(client_socket, client_address))
            client_thread.start()

        except Exception as e:
            print(f"Error while accepting connections: {e}")
            break

# 捕捉用戶輸入以關閉伺服器
def server_shutdown():
    while running:
        command = input("Type 'shutdown' to stop the server: ")
        if command.lower() == "shutdown":
            stop_server()
        else:
            try:
                print(con.execute(command).fetchall())
            except Exception as e:
                print(e)


# 開始接收連接並運行伺服器
def start_server():
    # 開啟兩個不同的執行緒：一個處理客戶端連接，另一個監控關閉指令
    threading.Thread(target=accept_connections, daemon=True).start()
    threading.Thread(target=server_shutdown, daemon=True).start()

    # 保持伺服器一直運行，直到手動停止
    while running:
        pass

# 開始伺服器
start_server()


C:\Users\nicholas\AppData\Local\Temp\ipykernel_10064\4222410871.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_str, conn)
C:\Users\nicholas\AppData\Local\Temp\ipykernel_10064\4222410871.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_str, conn)
C:\Users\nicholas\AppData\Local\Temp\ipykernel_10064\4222410871.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_str, conn)
C:\Users\nicholas\AppData\Local\Temp\ipykernel_10064\42224

port:1111
Server is listening on 127.0.0.1:1111...
Current online users: 1
Connection from ('127.0.0.1', 55350) has been established.
Client ('127.0.0.1', 55350) has selected the role: Member
displaysuccess

displaysuccess

[WinError 10053] 連線已被您主機上的軟體中止。
Connection with ('127.0.0.1', 55350) closed.
Current online users: 0
Type 'shutdown' to stop the server: shutdown

Shutting down the server...
Server socket closed.
Error while accepting connections: [WinError 10038] 嘗試操作的對象不是通訊端。
